In [2]:
!pip install pyspark

## Ways of Clusters : Spark Part 2


In part1 that is hosted https://www.kaggle.com/code/kamaljp/5ways-spark-waysof-partitioning discusses the way to divide and cluster the data. This notebook discusses how to read them.

In [3]:
#Creating the Spark Session

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession. \
    builder. \
    appName('Read_Data'). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/03 08:25:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Before we can Read

We have to find a dataset that can be used for reading into Spark Tables. Instead for searching for the new dataset, we are going to create a couple. Please follow part one for your reference

In [4]:
#importing the dataset

sales_raw = spark.read.csv('/kaggle/input/datasetbackups/dmart/Sales.csv',
                          inferSchema=True, sep='\t',header=True)

In [5]:
#Creating database so the Spark SQL tables can be created. 
spark.sql("CREATE DATABASE dmart_db")
spark.sql("USE dmart_db")

DataFrame[]

In [7]:
SQL = spark.sql
SQL("SET spark.sql.shuffle.partitions = 2;")

DataFrame[key: string, value: string]

In [ ]:
sales_raw.printSchema()

In [8]:
# Lets create a table that has partition builtin

sales_raw.write.saveAsTable("sales_partition",
                           mode='overwrite',
                           partitionBy='GMV',
                           format='parquet')

## We have the first one completed. Starting the second data

This will contain partitions based on two columns

In [9]:
#Lets take another file

first_sales = spark.read.csv("/kaggle/input/datasetbackups/dmart/firstfile.csv",
                            inferSchema=True,header=True,sep=',')

In [11]:
trimed_sales = first_sales.selectExpr("split_part(Date, ' ',1) as day_date",
                    "Sales_name", "gmv_new", "units")

In [ ]:
trimed_sales.write.saveAsTable("trimed_sales_partition",
                              mode='overwrite',
                              format='parquet',
                              partitionBy=['Sales_name','gmv_new'])

In [ ]:
!ls /kaggle/working/spark-warehouse/dmart_db.db/sales_partition/ | head -n 10

In [ ]:
SQL("SHOW PARTITIONS trimed_sales_partition").show(2,truncate=False)

In [ ]:
!ls /kaggle/working/spark-warehouse/dmart_db.db/trimed_sales_partition/

In [ ]:
!ls /kaggle/working/spark-warehouse/dmart_db.db/trimed_sales_partition/Sales_name\=BED | head -n 5

## The third one... Really!!!

In [ ]:
trimed_sales.write.bucketBy(10, 'Sales_name').saveAsTable("bucketed_sales")

In [ ]:
!ls /kaggle/working/spark-warehouse/dmart_db.db/bucketed_sales/ 

## Now we have the files. 

Lets talk about loading these files into the Spark Cluster as Tables

******* Example Code *********

CREATE TABLE student (id INT, 
                        name STRING, 
                         age INT)

USING CSV

PARTITIONED BY (age)

CLUSTERED BY (Id) INTO 4 buckets;

In [ ]:
#We will be using the spark read api for doing the same